In [1]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

In [3]:
def find_common_range(wavelength1, flux1, wavelength2, flux2):
    common_range = np.intersect1d(wavelength1, wavelength2)
    indices1 = np.searchsorted(wavelength1, common_range, side='left')
    indices2 = np.searchsorted(wavelength2, common_range, side='left')
    indices1 = np.clip(indices1, 0, len(flux1) - 1)
    indices2 = np.clip(indices2, 0, len(flux2) - 1)
    common_flux1 = flux1[indices1]
    common_flux2 = flux2[indices2]
    return common_range, common_flux1, common_flux2

# Load Transiting Spectrum:
transit_file = "K2-10_med-59296-TD112445N020547H01_sp08-229.fits"
data_transit = fits.open(transit_file)

# Red arm spectra
transit_wvr = data_transit[2].data["WAVELENGTH"]
transit_red = data_transit[2].data["FLUX"]

# Blue arm spectra
transit_wvb = data_transit[1].data["WAVELENGTH"]
transit_blue = data_transit[1].data["FLUX"]

# Calculate median flux for each arm
median_flux_red = np.median(transit_red)
median_flux_blue = np.median(transit_blue)

# Normalize spectra
normalized_red = transit_red / median_flux_red
normalized_blue = transit_blue / median_flux_blue

# Load Non-Transit Spectra:
non_transit_files = ["K2-10_med-59248-TD112445N020547H01_sp08-229.fits", "K2-10_med-59241-TD112445N020547H01_sp08-229.fits"]

non_transit_red_spectra = []
non_transit_blue_spectra = []

# Determine the minimum length among non-transit spectra
min_length_red = min(len(data_non_transit[2].data["FLUX"]) for file in non_transit_files)
min_length_blue = min(len(data_non_transit[1].data["FLUX"]) for file in non_transit_files)

for file in non_transit_files:
    data_non_transit = fits.open(file)
    non_transit_red_spectra.append(data_non_transit[2].data["FLUX"][:min_length_red])
    non_transit_blue_spectra.append(data_non_transit[1].data["FLUX"][:min_length_blue])

# Find common wavelength range for red and blue arms
common_range_red, transit_red_common, non_transit_red_common = find_common_range(transit_wvr, np.mean(non_transit_red_spectra, axis=0), transit_wvr, transit_red)
common_range_blue, transit_blue_common, non_transit_blue_common = find_common_range(transit_wvb, np.mean(non_transit_blue_spectra, axis=0), transit_wvb, transit_blue)

# Divide In-Transit Spectrum by Mean Out-of-Transit Spectrum:
divided_transit_red = transit_red_common / non_transit_red_common
divided_transit_blue = transit_blue_common / non_transit_blue_common

# Binning for Increased Signal-to-Noise Ratio (SNR):
def bin_spectrum(wavelength, flux, bin_size):
    binned_wavelength = wavelength[:len(wavelength)//bin_size * bin_size].reshape(-1, bin_size).mean(axis=1)
    binned_flux = flux[:len(flux)//bin_size * bin_size].reshape(-1, bin_size).mean(axis=1)
    return binned_wavelength, binned_flux

bin_size = 10

binned_wvb, binned_normalized_blue = bin_spectrum(common_range_blue, divided_transit_blue, bin_size)
binned_wvr, binned_normalized_red = bin_spectrum(common_range_red, divided_transit_red, bin_size)

# Plotting:
f, (ax, ax2) = plt.subplots(1, 2, sharey=True, facecolor='w', figsize=(14, 6))
f.subplots_adjust(wspace=0.020)

ax.plot(binned_wvb, binned_normalized_blue, label='Normalized Blue Arm', color='blue')
ax2.plot(binned_wvr, binned_normalized_red, label='Normalized Red Arm', color='red')

ax.plot([np.nan, np.nan], [np.nan, np.nan], color='k', linestyle='-', linewidth=1)
ax2.plot([np.nan, np.nan], [np.nan, np.nan], color='k', linestyle='-', linewidth=1)

ax.set_xlim(np.min(binned_wvb), np.max(binned_wvb))
ax2.set_xlim(np.min(binned_wvr), np.max(binned_wvr))

ax.set_ylim(np.min([binned_normalized_blue, binned_normalized_red]), np.max([binned_normalized_blue, binned_normalized_red]))
ax2.set_ylim(np.min([binned_normalized_blue, binned_normalized_red]), np.max([binned_normalized_blue, binned_normalized_red]))

ax.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax.yaxis.tick_left()
ax2.tick_params(labelright='off')
ax2.yaxis.tick_right()

f.text(0.5, 0.03, 'Wavelength [Angstrom]', ha='center', size=12)
f.text(0.065, 0.5, 'Normalized Flux', va='center', rotation='vertical', size=12)
f.suptitle('Normalized Spectra (Broken Axis)', size=14)

plt.show()


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 1) + inhomogeneous part.